# Проект SQL

## Описание проекта

Наша компания решила купить крупный сервис для чтения книг по подписке. В доступе имеется информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Цель проекта** — проанализировать базу данных, сформулировать ценностное предложение для нового продукта.

**Описание данных**

Таблица `books`

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors`

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers`

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings`

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews`

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Получение данных

In [1]:
# импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
 db_config['pwd'], 
 db_config['host'], 
 db_config['port'], 
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Обзор данных

In [2]:
# смотрим содержимое датасета books
query = """
select * 
from books
limit 5
"""

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# смотрим содержимое датасета authors
query = """
select * 
from authors
limit 5
"""

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# смотрим содержимое датасета publishers
query = """
select * 
from publishers
limit 5
"""

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# смотрим содержимое датасета ratings
query = """
select * 
from ratings
limit 5
"""

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# смотрим содержимое датасета reviews
query = """
select * 
from reviews
limit 5
"""

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Исследование данных

**Посчитаем, сколько книг вышло после 1 января 2000 года.**

In [7]:
query = """
SELECT COUNT(DISTINCT book_id)
FROM books
WHERE publication_date > '2000-01-01'
"""

q1 = pd.io.sql.read_sql(query, con = engine)

print('Количество книг, которые вышли после 1 января 2000 года:', q1['count'][0])

Количество книг, которые вышли после 1 января 2000 года: 819


**Вывод**: Всего у нас в каталоге 1000 книг, из которых 819 книг (82% от общего количества) вышло после 1 января 2000 года.

**Для каждой книги посчитаем количество обзоров и среднюю оценку.**

In [65]:
query = """
SELECT b.title, 
       COUNT(DISTINCT r.review_id) AS reviews, 
       ROUND(AVG(t.rating), 2) AS mean_rating
FROM books AS b
LEFT JOIN reviews AS r ON b.book_id=r.book_id
LEFT JOIN ratings AS t ON b.book_id=t.book_id
GROUP BY b.book_id
"""

q2 = pd.io.sql.read_sql(query, con = engine)
print(q2)

                                                 title  reviews  mean_rating
0                                         'Salem's Lot        2         3.67
1                   1 000 Places to See Before You Die        1         2.50
2    13 Little Blue Envelopes (Little Blue Envelope...        3         4.67
3    1491: New Revelations of the Americas Before C...        2         4.50
4                                                 1776        4         4.00
..                                                 ...      ...          ...
995           Wyrd Sisters (Discworld  #6; Witches #2)        3         3.67
996                        Xenocide (Ender's Saga  #3)        3         3.40
997                                    Year of Wonders        4         3.20
998                        You Suck (A Love Story  #2)        2         4.50
999  Zen and the Art of Motorcycle Maintenance: An ...        4         3.83

[1000 rows x 3 columns]


**Вывод**: у Сумерек наибольшее количество обзоров (7), а вот средний рейтинг ниже 4, тогда как книжки о Ходячих мертвецах отличаются хорошим рейтингом, но малым количеством обзоров (всего 2). В остальном всё зависит от книги.

**Определим издательство, которое выпустило наибольшее число книг толще 50 страниц (так мы исключим из анализа брошюры).**

In [9]:
query = """
SELECT p.publisher, 
       COUNT(DISTINCT b.book_id) AS book_count
FROM books AS b
LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY book_count DESC
LIMIT 1
"""

q3 = pd.io.sql.read_sql(query, con = engine)
print('Издательство:', q3['publisher'][0])
print('Количество книг:', q3['book_count'][0])

Издательство: Penguin Books
Количество книг: 42


**Вывод**: Penguin Books выпустило больше других книг толщиной 50+ страниц. Это больше чем в 2 раза превышает количество книг издателя, который закрывает топ-5 данного рейтинга. Стоит отметить, что это британская компания, которая публикует книги на английском языке.

**Определим автора с самой высокой средней оценкой книг — будем учитывать только книги с 50 и более оценками.**

In [24]:
query = """
WITH
i AS (SELECT a.author_id, 
             a.author, 
             ROUND(AVG(r.rating), 2) AS mean_rating
      FROM books b
      LEFT JOIN ratings AS r ON r.book_id=b.book_id
      LEFT JOIN authors AS a ON b.author_id=a.author_id
      GROUP BY a.author_id, b.book_id
      HAVING COUNT(r.rating_id) >= 50)
SELECT author,
       ROUND(AVG(mean_rating), 2) AS total_rating
FROM i
GROUP BY author
ORDER BY total_rating DESC
"""

q4 = pd.io.sql.read_sql(query, con = engine)

print('Имя автора:', q4['author'][0])
print('Средний рейтинг:', q4['total_rating'][0])

Имя автора: J.K. Rowling/Mary GrandPré
Средний рейтинг: 4.29


**Вывод**: Здесь мы видим гордость британской литературы, которая на протяжении многих лет пользуется популярностью у читателей.

**Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [63]:
query = """
WITH
i AS (SELECT r.username
      FROM ratings AS r
      LEFT JOIN books AS b ON r.book_id=b.book_id
      GROUP BY r.username
      HAVING COUNT(r.rating_id) > 50
),
k AS (SELECT r.username,
             COUNT(DISTINCT r.review_id) AS reviews_count
      FROM reviews AS r
      LEFT JOIN books AS b ON r.book_id=b.book_id
      GROUP BY r.username)
      
SELECT ROUND(AVG(k.reviews_count), 2) AS reviews_count_mean
FROM i LEFT JOIN k ON i.username=k.username
"""

q5 = pd.io.sql.read_sql(query, con = engine)

print('Среднее количество обзоров от пользователей, \nкоторые поставили больше 50 оценок:', q5['reviews_count_mean'][0])

Среднее количество обзоров от пользователей, 
которые поставили больше 50 оценок: 24.33


## Вывод

Мы произвели анализ баз данных нашего сервиса. Посчитали количество обзоров и среднюю оценку для всех книг. Выяснилось следующее:

* 82% от общего числа книг вышло после 1 января 2000 года. Это говорит о том, что большинство книг содержит актуальную информацию;
* Penguin Books - лидер среди издательств по количеству выпущенных книг;
* J.K. Rowling совместно с иллюстратором Mary GrandPré - лидеры по средней оценке популярных книг, т.е. книг с 50+ оценками;
* Среднее количество обзоров от пользователей, которые поставили 50+ оценок: 24.33